<a href="https://colab.research.google.com/github/kevinOriginal/news-recommender/blob/main/News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Big Data Term Project
- 2020 - 2 CAU Big Data Term Project
- News Recommendation System

import library

In [ ]:
from glob import glob
import math
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt 

## 1. Get Data

Get Crowling Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/BigData/newsdata/'
files = sorted(glob('./**/*.parquet' ,recursive=True))

In [ ]:
print(len(files))

## 2. Data Processing

- parquet 데이터를 pandas로 읽어와 필요한 feature만 가져온다.
- maskedId 의 **** 를 제거하고 idNo와 붙인다.

In [ ]:
data = []
for i in range (0, len(files)):
  df = pd.read_parquet(files[i])
  data_select = df.loc[:, ['idNo','maskedUserId','templateId','objectId','sympathyCount', 'antipathyCount']]

  # maskedId 뒤에 **** 제거
  id_split = data_select['maskedUserId'].str.split("*")
  process_id = id_split.str[0]
  data_select['maskedUserId'] = process_id

  data.append(data_select)
  print(i+1, " data is processing...")

- 각 날짜의 데이터 이어 붙이기

In [ ]:
result = pd.concat([data[0],data[1],data[2],data[3],data[4],data[5],data[6],
                    data[7],data[8],data[9],data[10],data[11],data[12],data[13]], ignore_index=True)

- 결측값 있는 행 제거하기

In [ ]:
# 결측값 데이터 조회 --> idNo에 169621개
result.isnull().sum()

In [ ]:
# 결측값 존재시 drop
notnull_result = result.dropna()  # NULL 값 제거
notnull_result = notnull_result[notnull_result.idNo != ''] # 공백 데이터 제거

In [ ]:
# 결측값이 제거된 것을 확인
notnull_result.isnull().sum()

In [ ]:
# userId 생성 (maskedUserId + idNo)
notnull_result["userId"] = notnull_result['maskedUserId'].map(str) + notnull_result['idNo']
notnull_result = notnull_result.reset_index(drop=True)

In [ ]:
# Sum of 좋아요 + 싫어요, 좋아요/Sum, 싫어요/Sum
notnull_result['like+hate'] = notnull_result['sympathyCount'] + notnull_result['antipathyCount']
notnull_result['likeRatio'] = notnull_result['sympathyCount'] / notnull_result['like+hate']
notnull_result['hateRatio'] = notnull_result['antipathyCount'] / notnull_result['like+hate']

In [ ]:
notnull_result.head() # 마지막 열에 추가된 것을 확인

- 좋아요/싫어요 ratio 가 0인 경우가 대다수 & null값도 많음,,

In [ ]:
notnull_result.isnull().sum() # null값 643918개

In [ ]:
sum(notnull_result['likeRatio'] == 0) # 0값 = 119825

In [ ]:
notnull_result.to_parquet(path + '/notnull_result.parquet')

## 3. 댓글 분포 visualization
- 가로 축: user의 댓글 수 (최소 1개 ~ 최대 238개)
- 세로 축: user수 (예: 2주 동안 댓글을 7개 단 user 수) 

In [ ]:
news_df = notnull_result.loc[:, ['userId','objectId']]
news_df = news_df.sort_values(by = ['userId'])
# userid로 묶어서 총 댓글 갯수 확인
news_df = news_df.groupby('userId').count()
news_df = news_df.sort_values(by = ['objectId'])
news_df['userId'] = news_df.index
news_group = news_df.set_index('objectId')
news_group = news_group.groupby('objectId').count()


plt.rcParams['figure.figsize'] = [25, 10]
bar = news_group.plot.bar(grid = False)
plt.xlabel('Reply news count')
plt.ylabel('User count')
plt.show()
plt.savefig('reply count')

In [ ]:
news_group

In [ ]:
# 댓글 100개 이상 단 유저 수
news_group[(news_group.index >= 10)].sum()

##3. Vector 만들기

In [ ]:
notnull_result

In [ ]:
# 결측값이 제거된 DataFrame에서 userid와 objectId 가져오기
user_news = notnull_result.loc[:, ['objectId','userId']]
user_news = user_news.sort_values(by = ['userId'])
user_news = user_news.reset_index(drop=True)
user_news

In [ ]:
# 같은 기사에 여러 개의 댓글을 다는 경우
user_news[user_news.duplicated(["objectId","userId"], keep=False)]

In [ ]:
# 한 사람이 같은 뉴스 기사에 여러 개의 댓글을 다는 경우 제거
user_news = user_news.drop_duplicates(["objectId","userId"], keep = "first")
user_news = user_news.set_index('userId')

In [ ]:
user_news

* one-hot encoding

In [ ]:
user_vector = pd.get_dummies(user_news['objectId'])
user_vector

In [ ]:
user_vector = user_vector.groupby(level=0).sum()

# parquet & csv로 변환
user_vector.to_parquet(path + '/user_vector.parquet')
user_vector.to_csv(path + '/user_vector_index.csv', header=True, index= True)
user_vector.to_csv(path + '/user_vector.csv', header=False, index= False)

## Normalization

In [ ]:
# 코랩 경로에 user_vector.parquet 넣기
vector = pd.read_parquet('/content/user_vector.parquet')
norm_vector = vector[vector.sum(axis = 1) >= 10] # 10보다 크면 TRUE, TRUE 값만 가지고 오기
norm_vector.to_parquet('/content/norm_vector.parquet')